<a href="https://colab.research.google.com/github/Lakshman3556/Machine_Learning/blob/main/ML__week4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

df=pd.read_csv('/content/drive/MyDrive/ML/titanic_train.csv')
print(df.head())
df.shape

   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   NaN        S  


(891, 12)

In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Load dataset
df = pd.read_csv('/content/drive/MyDrive/ML/titanic_train.csv')

# Drop columns that are not useful for modeling
df = df.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1)

# Handle missing values
df['Age'] = df['Age'].fillna(df['Age'].median())
df['Embarked'] = df['Embarked'].fillna(df['Embarked'].mode()[0])

# Encode categorical variables
label_enc = LabelEncoder()
df['Sex'] = label_enc.fit_transform(df['Sex'])        # male=1, female=0
df['Embarked'] = label_enc.fit_transform(df['Embarked'])  # C=0, Q=1, S=2

# Features and target
X = df.drop('Survived', axis=1)
y = df['Survived']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=0
)


In [21]:
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import roc_auc_score

from mlxtend.feature_selection import SequentialFeatureSelector as SFS

X_train, X_test, y_train, y_test = train_test_split(
    df.drop(labels=['Survived'], axis=1),
    df['Survived'],
    test_size=0.3,
    random_state=0)
def correlation(dataset, threshold):
    col_corr = set()  # Set of all the names of correlated columns
    corr_matrix = dataset.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i, j]) > threshold: # we are interested in absolute coeff value
                colname = corr_matrix.columns[i]  # getting the name of column
                col_corr.add(colname)
    return col_corr
corr_features = correlation(X_train, 0.8)
print('correlated features: ', len(set(corr_features)) )





correlated features:  0


In [22]:
X_train.drop(labels=corr_features, axis=1, inplace=True)
X_test.drop(labels=corr_features, axis=1, inplace=True)
X_train.shape, X_test.shape

((623, 7), (268, 7))

In [24]:
sfs1 = SFS(RandomForestClassifier(n_jobs=4),   #  4 CPU cores in parallel (faster computation)
           k_features=6,   #select the best 5 features
           forward=True,  #adds one feature at a time , False for SBS
           #floating=False,   (True:  after each addition, it checks whether any previously added feature can be removed to 			   improve the model score)
           verbose=2,  #progress printed in the console.
           scoring='r2',
           cv=3)
sfs1 = sfs1.fit(np.array(X_train.fillna(0)), y_train)

# Print the results
print('Best accuracy score: %.2f' % sfs1.k_score_)   # k_score_ shows the best score
print('Best subset (indices):', sfs1.k_feature_idx_) # k_feature_idx_ shows the index of features
                                                    # that yield the best score
print('Best subset (corresponding names):', sfs1.k_feature_names_) # k_feature_names_ shows the feature names


[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:    4.4s finished

[2025-08-13 09:33:05] Features: 1/6 -- score: 0.10169024010887551[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    4.2s finished

[2025-08-13 09:33:09] Features: 2/6 -- score: 0.12880415411036608[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    3.3s finished

[2025-08-13 09:33:12] Features: 3/6 -- score: 0.1423868312757204[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    2.5s finished

[2025-08-13 09:33:15] Features: 4/6 -- score: 0.1215498525647259[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    1.9s finished

[2025-08-13 09:33:17] Features: 5/6 -- score: 0.20273727358154325

Best accuracy score: 0.17
Best subset (indices): (0, 1, 2, 3, 4, 6)
Best subset (corresponding names): ('0', '1', '2', '3', '4', '6')


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    1.4s finished

[2025-08-13 09:33:18] Features: 6/6 -- score: 0.16883202099737551